In [1]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
from surprise import Dataset, Reader, KNNWithMeans, SVD, accuracy
from surprise.model_selection import cross_validate, train_test_split


## Reading Data

In [2]:
reviews_train = pd.read_csv('../data/reviews_train.csv', usecols=['RecipeId', 'AuthorId', 'Rating'])
reviews_test = pd.read_csv('../data/reviews_test.csv', usecols=['RecipeId', 'AuthorId', 'Rating'])


## Constructing base model

### SVD model

In [3]:
svd = SVD()
reader = Reader(rating_scale=(1,5))
# Loads Pandas dataframe
data = Dataset.load_from_df(reviews_train.loc[:10000, ["AuthorId", "RecipeId", "Rating"]], reader)

In [5]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5896  0.6721  0.6078  0.5768  0.6328  0.6158  0.0338  
MAE (testset)     0.4218  0.4554  0.4319  0.4239  0.4403  0.4347  0.0123  
Fit time          0.19    0.27    0.23    0.19    0.17    0.21    0.04    
Test time         0.07    0.02    0.01    0.02    0.01    0.03    0.02    


In [13]:
a['test_rmse'].mean()

0.6158185612609113

In [22]:

svd.fit(trainset)
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.6007


0.6006939724617784

In [33]:
svd.predict(1000,2000)

Prediction(uid=1000, iid=2000, r_ui=None, est=4.69875, details={'was_impossible': False})

### KNN model

In [28]:
reader = Reader(rating_scale=(1, 5))
# Loads Pandas dataframe
data_train = Dataset.load_from_df(reviews_train.loc[:10000, ["AuthorId", "RecipeId", "Rating"]], reader)
data_test = Dataset.load_from_df(reviews_test.loc[:100, ["AuthorId", "RecipeId", "Rating"]], reader)

# To use user-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
trainingSet = data_train.build_full_trainset()
testingSet = data_test.build_full_trainset()
algo = KNNWithMeans(sim_options=sim_options)

In [29]:
# fitting the model
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [31]:
#prediction = algo.predict(1634, 5466)
algo.test(testingSet)

TypeError: 'Trainset' object is not iterable

In [49]:
reviews_test[reviews_test['AuthorId']==2946]

,RecipeId,AuthorId,Rating


## Parameter Tuning

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])